# Dados e modelos

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import joblib

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv("/content/imdb-reviews-pt-br.csv")
df['sentiment'] = df['sentiment'].replace({'pos': 'Positivo', 'neg': 'Negativo'})

In [ ]:
df

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",Negativo
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,Negativo
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",Negativo
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,Negativo
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,Negativo
...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",Positivo
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,Positivo
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,Positivo
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,Positivo


In [ ]:
X = df['text_pt']
y = df['sentiment']

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# positivo=1, negativo=0
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'), max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Avaliando diferentes modelos

In [ ]:
# Lista de modelos a serem avaliados
models = {
    "LinearSVC": LinearSVC(random_state=42),
    "LogisticRegression": LogisticRegression(random_state=42, max_iter=1000),
    "RandomForest": RandomForestClassifier(random_state=42, n_estimators=100),
    "MultinomialNB": MultinomialNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
}

# Avaliar o desempenho de cada modelo
results = []
for model_name, model in models.items():
    # Treinar o modelo
    model.fit(X_train_tfidf, y_train_encoded)
    # Fazer previsões
    y_pred_encoded = model.predict(X_test_tfidf)
    # Avaliar o desempenho
    report = classification_report(y_test_encoded, y_pred_encoded, output_dict=True)
    accuracy = report['accuracy']
    f1_score = report['weighted avg']['f1-score']
    results.append({"Model": model_name, "Accuracy": accuracy, "F1-Score": f1_score})

# Converter resultados em DataFrame para exibição
results_df = pd.DataFrame(results)

In [ ]:
results_df

,Model,Accuracy,F1-Score
0,LinearSVC,0.870939,0.870921
1,LogisticRegression,0.879094,0.879060
2,RandomForest,0.835625,0.835594
3,MultinomialNB,0.856315,0.856295
4,KNN,0.737768,0.736834


# Otimização de hiperparâmetros

In [ ]:
param_grid = {
    'C': [0.01, 0.1, 1, 10],  # Regularização
    'penalty': ['l2', 'l1'],  # Penalidade
    'solver': ['liblinear', 'saga'],  # Solvers que suportam L1 e L2
}

# Instanciar o modelo de Logistic Regression
logistic_model = LogisticRegression(max_iter=1000, random_state=42)

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=logistic_model, param_grid=param_grid, cv=3, scoring='f1_weighted', verbose=1, n_jobs=-1)

# Realizar a busca pelos melhores hiperparâmetros
grid_search.fit(X_train_tfidf, y_train_encoded)

# Melhor modelo e hiperparâmetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

best_params

Fitting 3 folds for each of 16 candidates, totalling 48 fits


{'C': 1, 'penalty': 'l2', 'solver': 'saga'}

# Modelo final

In [ ]:
best_params = {'C': 1, 'penalty': 'l2', 'solver': 'saga'}

# Train the Logistic Regression model with the best parameters
final_model = LogisticRegression(**best_params, max_iter=1000, random_state=42)
final_model.fit(X_train_tfidf, y_train_encoded)

LogisticRegression(C=1, max_iter=1000, random_state=42, solver='saga')

In [ ]:
y_pred_encoded = final_model.predict(X_test_tfidf)
print(classification_report(y_test_encoded, y_pred_encoded, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

    Negativo       0.89      0.86      0.88      7430
    Positivo       0.87      0.90      0.88      7408

    accuracy                           0.88     14838
   macro avg       0.88      0.88      0.88     14838
weighted avg       0.88      0.88      0.88     14838



# Salvando os modelos

In [ ]:
model_path = "sentiment_classifier_model.pkl"
vectorizer_path = "tfidf_vectorizer.pkl"
label_encoder_path = "label_encoder.pkl"

joblib.dump(final_model, model_path)
joblib.dump(vectorizer, vectorizer_path)
joblib.dump(label_encoder, label_encoder_path)

['label_encoder.pkl']

# Reusando os modelos

In [ ]:
# Caminhos para os arquivos salvos
model_path = 'sentiment_classifier_model.pkl'
vectorizer_path = 'tfidf_vectorizer.pkl'
label_encoder_path = 'label_encoder.pkl'

# Carregar os arquivos salvos
classifier = joblib.load(model_path)
vectorizer = joblib.load(vectorizer_path)
label_encoder = joblib.load(label_encoder_path)

# Novo texto para inferência
# Inference example
new_text = ["O filme foi maravilhoso, cheio de emoção e uma ótima história!"]
new_text_tfidf = vectorizer.transform(new_text)
predicted_label_encoded = classifier.predict(new_text_tfidf)
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)
print(f"Texto: {new_text[0]}")
print(f"Sentimento previsto: {predicted_label[0]}")

Texto: O filme foi maravilhoso, cheio de emoção e uma ótima história!
Sentimento previsto: Positivo
